In [ ]:
"""
Local search algorithms for dynamic replanning
"""

import random
import math
from typing import List, Tuple, Dict, Callable
from ..utils import Node, manhattan_distance, calculate_path_cost

def hill_climbing(environment, start: Tuple[int, int], goal: Tuple[int, int],
                 max_iterations: int = 1000) -> Dict[str, any]:
    """Hill Climbing with random restarts for path optimization"""

    def generate_random_path(current_pos, steps=20):
        """Generate a random valid path from current position"""
        path = [current_pos]
        current = current_pos

        for _ in range(steps):
            neighbors = environment.get_neighbors(*current)
            if not neighbors:
                break
            next_pos = random.choice(neighbors)
            path.append((next_pos[0], next_pos[1]))
            current = (next_pos[0], next_pos[1])

            if current == goal:
                break

        return path

    best_path = None
    best_cost = float('inf')

    for restart in range(10):  # Random restarts
        current_path = generate_random_path(start)
        current_cost = calculate_path_cost(current_path, environment)

        for iteration in range(max_iterations // 10):
            # Generate neighbor by modifying the path
            if len(current_path) > 2:
                modify_index = random.randint(1, len(current_path) - 2)
                new_segment = generate_random_path(current_path[modify_index - 1], steps=3)
                new_path = current_path[:modify_index] + new_segment + current_path[modify_index + 2:]
            else:
                new_path = generate_random_path(start)

            new_cost = calculate_path_cost(new_path, environment)

            if new_cost < current_cost and new_path[-1] == goal:
                current_path, current_cost = new_path, new_cost

        if current_cost < best_cost and current_path[-1] == goal:
            best_path, best_cost = current_path, current_cost

    return {
        'path': best_path if best_path else [],
        'cost': best_cost,
        'nodes_expanded': max_iterations,
        'success': best_path is not None
    }

def simulated_annealing(environment, start: Tuple[int, int], goal: Tuple[int, int],
                       max_iterations: int = 1000) -> Dict[str, any]:
    """Simulated Annealing for dynamic obstacle avoidance"""

    current_path = [start]
    current = start

    # Build a simple path first
    while current != goal and len(current_path) < 100:
        neighbors = environment.get_neighbors(*current)
        if not neighbors:
            break

        # Prefer moving toward goal
        best_neighbor = None
        best_score = float('inf')

        for nx, ny, cost in neighbors:
            score = manhattan_distance((nx, ny), goal) + cost
            if score < best_score:
                best_score = score
                best_neighbor = (nx, ny, cost)

        if best_neighbor:
            current = (best_neighbor[0], best_neighbor[1])
            current_path.append(current)

    current_cost = calculate_path_cost(current_path, environment)
    best_path, best_cost = current_path[:], current_cost

    temperature = 1.0
    cooling_rate = 0.95

    for iteration in range(max_iterations):
        temperature *= cooling_rate

        if temperature < 0.1:
            break

        # Modify the path
        if len(current_path) > 3:
            modify_index = random.randint(1, len(current_path) - 2)

            # Try to find a bypass
            start_bypass = current_path[modify_index - 1]
            end_bypass = current_path[modify_index + 1] if modify_index + 1 < len(current_path) else goal

            # Simple bypass attempt
            bypass_found = False
            for nx, ny, cost in environment.get_neighbors(*start_bypass):
                if (nx, ny) != current_path[modify_index] and environment.is_valid_position(nx, ny):
                    for nnx, nny, ncost in environment.get_neighbors(nx, ny):
                        if (nnx, nny) == end_bypass:
                            new_path = current_path[:modify_index] + [start_bypass, (nx, ny), end_bypass] + current_path[modify_index + 2:]
                            new_cost = calculate_path_cost(new_path, environment)

                            # Accept worse solution with probability based on temperature
                            if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / temperature):
                                current_path, current_cost = new_path, new_cost

                                if new_cost < best_cost:
                                    best_path, best_cost = new_path[:], new_cost

                            bypass_found = True
                            break
                    if bypass_found:
                        break

    return {
        'path': best_path,
        'cost': best_cost,
        'nodes_expanded': max_iterations,
        'success': best_path[-1] == goal if best_path else False
    }